**WebScraping**  
 *Import*

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
import os
import time

In [2]:
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import pandas as pd

**On test le lien**

In [3]:
r = requests.get("https://freeglisse.com/fr/12-ski-occasion?page=1")
print(r.status_code)

200


**Scraper tous les url de toutes les pages du site**

In [33]:
def scrape_product_links(base_url):
    all_links = []
    page_num = 1
    while True:
        page_url = f"{base_url}?page={page_num}"  # Modifiez le schéma d'URL selon votre site

        # Obtenez le contenu HTML de la page
        response = requests.get(page_url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')

            # Trouvez toutes les balises h2 avec la classe "h3 product-title"
            product_titles = soup.find_all('h2', class_='h3 product-title')

            # Si aucune balise n'est trouvée, cela signifie que nous avons atteint la fin des pages
            if not product_titles:
                break

            # Extrayez les liens de ces balises et ajoutez-les à la liste de tous les liens
            for title in tqdm(product_titles, desc="scrape_product_links", unit='title'):
                link = title.find('a')['href']
                all_links.append(link)

            # Passez à la page suivante
            page_num += 1
        else:
            # Arrêtez la boucle si une erreur se produit lors de la requête
            print(f"Failed to retrieve page {page_num}. Status code: {response.status_code}")
            break

    return all_links

In [ ]:
# Exemple d'utilisation de la fonction
base_url = 'https://freeglisse.com/fr/12-ski-occasion'  # Remplacez par l'URL de votre site
product_links = scrape_product_links(base_url)
product_links

#Affichez tous les liens récupérés
for link in tqdm(product_links):
    print(link)

**Scrape les url d'une seule page**

In [4]:
def scrape_product_link(base_url):
    all_link = []

    # Obtenez le contenu HTML de la page
    response = requests.get(base_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        # Trouvez toutes les balises h2 avec la classe "h3 product-title"
        product_titles = soup.find_all('h2', class_='h3 product-title')

        # Extrayez les liens de ces balises et ajoutez-les à la liste de tous les liens
        for title in product_titles:
            link = title.find('a')['href']
            all_link.append(link)

    return all_link

In [ ]:
base_url = 'https://freeglisse.com/fr/12-ski-occasion?page=8'  # Remplacez par l'URL de votre site
product_link = scrape_product_link(base_url)
product_link

**Scraper les stocks**  
Fonction d'un bot permettant de scraper les stocks par quantité et taille de skis

In [15]:
def products(url, quality_xpath, base_xpath, button_xpath, availability_xpath):

    driver = webdriver.Chrome() # mise en place du Webdriver 

    all_products_availability = {}

    try:
        driver.get(url)

        # Attendez que les éléments de qualité soient chargés
        qualities = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, f"{quality_xpath}/li")))
        
        for quality_index in range(1, len(qualities) + 1):

            # Sélectionner la qualité
            quality_li_xpath = f"{quality_xpath}/li[{quality_index}]" # va chercher l'index du bouton qualité pour incrémenter et permettre une recherche sur l'indexation
            quality_li_element = driver.find_element(By.XPATH, quality_li_xpath) # cherche l'élement sur lequel on souhaite intéragir (en l'occurence le bouton qualité)
            quality_text = quality_li_element.find_element(By.XPATH, "./label/span").text # va scraper le text du bouton qualité par le biais du XPATH 
            quality_li_element.click() # Clique sur le bouton qualité
            
            # Laissez du temps pour que la sélection de la qualité se mette à jour
            time.sleep(1)  
            product_availability = {}
            
            # Attendez que les éléments de taille soient rechargés pour la qualité sélectionnée
            sizes = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, f"{base_xpath}/li")))
            
            for index in range(1, len(sizes) + 1):
                li_xpath = f"{base_xpath}/li[{index}]" # va chercher l'index du bouton taille pour incrémenter et permettre une recherche sur l'indexation
                li_element = driver.find_element(By.XPATH, li_xpath) # cherche l'élement sur lequel on souhaite intéragir (en l'occurence le bouton taille)
                
                if 'instock' in li_element.get_attribute('class'): # selectionne la taille seulement si 'instock' est dans le XPATH
                    size_text = li_element.find_element(By.XPATH, "./label/span").text # va scraper le text du bouton taille par le biais du XPATH 
                    li_element.click() # Clique sur le bouton taille

                    # Attendre que la page réagisse à la sélection de taille
                    time.sleep(1) 
                    count = 0 # initialisation du compteur de clique (qui sera pris en considération pour calculer le stock)
                    
                    while True:
                        availability_message = driver.find_element(By.XPATH, availability_xpath).text # va scraper le text 'disponible' par le biais du XPATH 
                        
                        if "Disponible" in availability_message: # si message disponible apparait, il continu de cliquer
                            driver.find_element(By.XPATH, button_xpath).click() # clique sur le bouton pour augmenter la quantité de ski
                            count += 1 #incrémente le compteur 
                            time.sleep(1)  # Attendre la réaction de la page au clic
                        else:
                            break 

                    product_availability[size_text] = count
                    time.sleep(1)  # Pause avant de passer à la taille suivante
            
            all_products_availability[quality_text] = product_availability

    except Exception as e:
        print(f"Une erreur s'est produite : {e}")

    finally:
        driver.quit()
    return {url: all_products_availability}

**Scraping des urls de toutes les pages**  

In [ ]:
base_url = 'https://freeglisse.com/fr/12-ski-occasion'  
product_link = scrape_product_links(base_url)
product_link

**Scraping des stocks selon qualité et taille de toutes les pages du site**

In [ ]:
quality_xpath = '/html/body/main/section/div/div/div/section/div[1]/div[2]/div[3]/div[2]/form/div[1]/div[1]/ul'
sizes_xpath = '/html/body/main/section/div/div/div/section/div[1]/div[2]/div[3]/div[2]/form/div[1]/div[2]/ul'
button_xpath = '//*[@id="add-to-cart-or-refresh"]/div[2]/div[1]/div[1]/div/span[3]/button[1]/i'
availability_xpath = '/html/body/main/section/div/div/div/section/div[1]/div[2]/div[3]/div[2]/form/div[2]/span/span'
stock_total = []

for link in tqdm(product_link, desc='products', unit='link'):
    stock = products(link, quality_xpath, sizes_xpath, button_xpath, availability_xpath)
    stock_total.append(stock)

stock_total

**Création du CSV avec les résultats des stocks**

In [44]:
import csv


# Définir le nom du fichier CSV de sortie
csv_file = 'stocks.csv'

# Ouvrir le fichier CSV en mode écriture
with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)

    # Écrire les en-têtes
    writer.writerow(['URL', 'Qualité', 'Taille', 'Quantité'])

    # Parcourir les données
    for item in results:
        for url, quality_info in item.items():
            url_column = url

            # Parcourir les informations sur la qualité
            for quality, size_info in quality_info.items():
                quality_column = quality

                # Parcourir les informations sur la taille et la quantité
                for size, quantity in size_info.items():
                    # Écrire une ligne dans le fichier CSV
                    writer.writerow([url_column, quality_column, size, quantity])

print(f"Les données ont été écrites dans {csv_file}")


Les données ont été écrites dans stocks.csv


**Affichage du Dataframe**

In [47]:
df= pd.read_csv('stocks.csv')
df

,URL,Qualité,Taille,Quantité
0,https://freeglisse.com/fr/ski-occasion-adulte-...,Qualité A,156 cm,16
1,https://freeglisse.com/fr/ski-occasion-adulte-...,Qualité A,164 cm,14
2,https://freeglisse.com/fr/ski-occasion-adulte-...,Qualité A,172 cm,9
3,https://freeglisse.com/fr/ski-de-fond-occasion...,Qualité C,150 cm,11
4,https://freeglisse.com/fr/ski-de-fond-occasion...,Qualité C,160 cm,12
...,...,...,...,...
219,https://freeglisse.com/fr/ski-occasion-junior-...,Qualité A,120 cm,3
220,https://freeglisse.com/fr/ski-occasion-junior-...,Qualité A,110 cm,3
221,https://freeglisse.com/fr/ski-occasion-junior-...,Qualité B,130 cm,3
222,https://freeglisse.com/fr/ski-de-rando-occasio...,Qualité B,178 cm,3
